<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Importing the libraries</h3>

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import cvxpy as cp
# using sklearn.svm SVC package
from sklearn import svm

<h3 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Importing the dataset</h3>

In [2]:
# Importing the training and test data set
train_dataset = pd.read_csv("train.csv", header=None)
test_dataset = pd.read_csv("test.csv", header=None)

In [3]:
# Dividing the first 4000 samples as X_train and y_train from train dataset
X_train = train_dataset.iloc[:4000, 1:].values
y_train = train_dataset.iloc[:4000, 0].values

In [4]:
# Taking the values after 4000 samples. Total values in training dataset = 8500
# We allocated first 4000 samples to X_train and y_train
# Now remaining 4500 samples are assigned for validation set
X_val = train_dataset.iloc[4000:, 1:].values
y_val = train_dataset.iloc[4000:, 0].values

In [5]:
# Dividing test set into X_test and y_test
X_test = test_dataset.iloc[:, 1:].values
y_test = test_dataset.iloc[:, 0].values

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Changing y from [0, 1] to [-1, 1]</h5>

In [6]:
"""This is important because if we miss any of the y_test and y_val to convert it 
into -1, 1 the accuracy will hinder"""
y_train[y_train==0] = -1
y_test[y_test==0] = -1
y_val[y_val==0] = -1

In [7]:
num_samples, num_features = np.shape(X_train)

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 1</h5>

In [8]:
# custom function to train the model
def svm_train_primal(data_train, label_train, regularisation_para_C):
    num_samples, num_features = np.shape(data_train)
    W = cp.Variable(num_features)
    Psi = cp.Variable(num_samples)
    b = cp.Variable()
    C = regularisation_para_C
    
    # according to the formula given in the question notice C/num_samples
    objective = cp.Minimize(0.5 * cp.square(cp.norm(W)) + ((C/num_samples) * cp.sum(Psi)))
    constraints = [cp.multiply(label_train, data_train @ W + b) - 1 + Psi >= 0, Psi >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return {
        "prob":prob, 
        "W":W, 
        "b":b,
        "Psi": Psi
    }

In [9]:
# custom function to predict the values and then return accuracy
def svm_predict_primal(data_test, label_test, svm_model):
    result = svm_model.get("W").value @ data_test.T + svm_model.get("b").value
    y_pred = np.sign(result)
    total = len(label_test)
    correct = 0
    # calculating the correct values
    for idx, p in enumerate(label_test):
        if p == y_pred[idx]:
            correct += 1
    # accuracy
    return correct / total

### Training using primal form

In [10]:
# setting C = 100
svm_model = svm_train_primal(X_train, y_train, 100)

In [11]:
W_primal = svm_model.get("W")
b_primal = svm_model.get("b")
Psi_primal = svm_model.get("Psi")

# print("Optimal value: ", svm_model.get("prob").value)
print("**********Primal Form***********")
print("Status of optimization: ", svm_model.get("prob").status)
print("Sum of Optimal value of W for C = 100: ", np.sum(W_primal.value))
print("Optimal value of b for C = 100: ", b_primal.value)


**********Primal Form***********
Status of optimization:  optimal
Sum of Optimal value of W for C = 100:  -0.1452156803361282
Optimal value of b for C = 100:  1.779813717087077


In [12]:
# calculating the test accuracy for C = 100
primal_test_accuracy = svm_predict_primal(X_test, y_test, svm_model)

In [13]:
print(f"Test Accuracy for primal using user defined svm: {primal_test_accuracy}")

Test Accuracy for primal using user defined svm: 0.968


<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Sanity check with sklearn.svm SVC</h5>

In [14]:
sklearn_svm = svm.SVC(C = 100/num_samples, kernel='linear')
sklearn_svm.fit(X_train, y_train)

X_test_sanity = X_test.reshape(-1, num_features)
svm_y = sklearn_svm.predict(X_test_sanity)
print(f"Test Accuracy for primal using SVM: {sklearn_svm.score(X_test_sanity, y_test)}")

Test Accuracy for primal using SVM: 0.968


<span style = "font-size:20px; font-family:Garamond ; font-weight : normal; color : #fe346e; text-align: center; border-radius: 100px 100px;">Note: You can see that the accuracy of primal form and sklearn.svm SVC is same</span>

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 2</h5>

In [15]:
# Custom dual form 
def svm_train_dual(data_train, label_train, regularisation_para_C):
    num_samples, num_features = np.shape(data_train)
    alpha = cp.Variable(num_samples)
    # X prime, this will also be used later in calculating W
    x_dash = data_train * label_train.reshape(-1, 1)
    C = regularisation_para_C
    dualObjective = cp.Maximize(cp.sum(alpha) - (0.5 * cp.sum_squares(alpha@x_dash)))
    
    constraints = [alpha >= 0, alpha <= C/num_samples, cp.sum(alpha.T@label_train) == 0] 
    prob = cp.Problem(dualObjective, constraints)
    prob.solve()
    return {
        "prob": prob,
        "alpha": alpha,
        "x_dash": x_dash,
        "optimal_c": C/num_samples
    }

In [16]:
# setting C = 100
svm_dual_model = svm_train_dual(X_train, y_train, 100)
print("*********** Dual Form *************")
print("Status of optimization: ", svm_dual_model.get("prob").status)
print("Optimal value of alpha: ", np.sum(svm_dual_model.get("alpha").value))

*********** Dual Form *************
Status of optimization:  optimal
Optimal value of alpha:  7.2912583245569005


### Finding W and b from dual form using alpha with +- 10% margin
- This is done to compare the values of W and b with primal form to make sure our implementation is correct

In [17]:
# finding W
# x_dash is y_train * x_train and w is sum(alpha * x_dash)
alpha = svm_dual_model.get("alpha")
x_dash = svm_dual_model.get("x_dash")
optimal_c = svm_dual_model.get("optimal_c")
converted_W = np.multiply(x_dash.T, alpha.value)

In [18]:
print("optimal W value: ", np.sum(converted_W))

optimal W value:  -0.14015434253764303


In [19]:
# finding b
b = 0
count_instances = 0
for idx, a in enumerate(alpha.value):
    if (a>0) and (a<optimal_c):
        b += y_train[idx] - np.sum(converted_W.T @ X_train[idx])
        count_instances += 1

In [20]:
print(f"Value of b is {b / count_instances}")

Value of b is 1.7552444689510422


<span style = "font-size:20px; font-family:Garamond ; font-weight : normal; color : #fe346e; text-align: center; border-radius: 100px 100px;">Note: We can see that the values for W and b from dual form is approximately similar to the values in primal form. +- 10% margin on the values is ok</span>

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 3</h5>

In [21]:
# finding the support vectors for primal form
def get_primal_support_vectors(data_train, label_train, W, b, Psi):
    support_vectors = []
    primal_th = np.multiply(label_train, data_train @ W + b) - 1 + Psi
    for idx, th in enumerate(primal_th):
        if th <= 1e-4:
            support_vectors.append(data_train[idx])
    return support_vectors

In [22]:
# We get W_primal b_primal and Psi_primal when we did training on primal form
primal_support_vectors = get_primal_support_vectors(X_train, y_train, W_primal.value, b_primal.value, Psi_primal.value)

In [23]:
len(primal_support_vectors)

392

In [24]:
# primal support vectors
primal_support_vectors

[array([-0.36, -0.91, -0.99, -0.57, -1.38, -1.54, -1.64,  1.29,  0.65,
        -0.75,  1.11, -0.04,  1.86,  0.76, -0.67,  0.75,  0.07,  0.31,
         1.23, -0.7 , -0.31, -1.48,  0.11, -1.18,  0.78,  0.48,  0.85,
         2.11, -1.45,  0.74, -0.54,  1.31,  0.64, -1.42,  1.22, -1.35,
         0.4 , -0.72, -0.79,  0.8 ,  0.32, -0.76,  0.22,  2.59, -0.64,
         1.27,  0.66,  1.8 ,  0.77,  1.13, -1.69, -1.31,  1.66, -0.26,
        -0.17, -0.41, -0.8 ,  1.24, -1.51, -1.45, -1.07, -1.38, -0.92,
        -1.26, -2.02,  0.58, -0.58, -1.06, -0.05,  0.29, -1.72, -0.26,
        -1.67, -0.07,  0.81,  0.01, -0.22,  1.72,  0.43,  0.92, -0.68,
        -0.98, -1.12, -0.81, -0.79,  1.72, -1.33, -1.08, -1.37, -1.75,
        -0.9 , -0.76, -1.33,  0.37,  0.25,  0.24,  0.33, -0.72, -1.05,
        -0.47, -0.44, -0.89,  0.01, -1.29, -1.79, -0.39, -1.03,  0.29,
         1.2 ,  0.62, -0.19, -1.3 , -0.29,  2.5 ,  2.01,  0.75, -0.53,
         1.61, -1.62, -0.33, -0.93, -0.79,  0.89, -0.64,  0.13,  0.97,
      

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 4</h5>

In [25]:
# geting the dual support vectors
def get_dual_support_vectors(data_train, label_train, alpha):
    support_vectors = list()
    for idx, a in enumerate(alpha):
        if a >= 1e-4:
            support_vectors.append(data_train[idx])
    return support_vectors

In [26]:
# We got alpha.value while training on dual form
dual_support_vectors = get_dual_support_vectors(X_train, y_train, alpha.value)

In [27]:
len(dual_support_vectors)

391

In [28]:
# Dual Support Vectors
dual_support_vectors

[array([-0.36, -0.91, -0.99, -0.57, -1.38, -1.54, -1.64,  1.29,  0.65,
        -0.75,  1.11, -0.04,  1.86,  0.76, -0.67,  0.75,  0.07,  0.31,
         1.23, -0.7 , -0.31, -1.48,  0.11, -1.18,  0.78,  0.48,  0.85,
         2.11, -1.45,  0.74, -0.54,  1.31,  0.64, -1.42,  1.22, -1.35,
         0.4 , -0.72, -0.79,  0.8 ,  0.32, -0.76,  0.22,  2.59, -0.64,
         1.27,  0.66,  1.8 ,  0.77,  1.13, -1.69, -1.31,  1.66, -0.26,
        -0.17, -0.41, -0.8 ,  1.24, -1.51, -1.45, -1.07, -1.38, -0.92,
        -1.26, -2.02,  0.58, -0.58, -1.06, -0.05,  0.29, -1.72, -0.26,
        -1.67, -0.07,  0.81,  0.01, -0.22,  1.72,  0.43,  0.92, -0.68,
        -0.98, -1.12, -0.81, -0.79,  1.72, -1.33, -1.08, -1.37, -1.75,
        -0.9 , -0.76, -1.33,  0.37,  0.25,  0.24,  0.33, -0.72, -1.05,
        -0.47, -0.44, -0.89,  0.01, -1.29, -1.79, -0.39, -1.03,  0.29,
         1.2 ,  0.62, -0.19, -1.3 , -0.29,  2.5 ,  2.01,  0.75, -0.53,
         1.61, -1.62, -0.33, -0.93, -0.79,  0.89, -0.64,  0.13,  0.97,
      

<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 5</h5>

### Choosing optimal C by using validation set and then using it to predict values for test set

In [29]:
# Optimal C in case of prime model
C_list = [2**(-10), 2**(-8), 2**(-6), 2**(-4), 2**(-2), 2**0, 2**2, 2**4, 2**6, 2**8, 2**10]
primal_test_accuracy_list = []
for c in C_list:
    primal_model = svm_train_primal(X_train, y_train, c)
    test_accuracy = svm_predict_primal(X_val, y_val, primal_model)
    primal_test_accuracy_list.append(test_accuracy)
maxAccuracy = max(primal_test_accuracy_list)
maxAccuracyIdx = primal_test_accuracy_list.index(maxAccuracy)
optimal_C = C_list[maxAccuracyIdx]

In [30]:
print(f"Optimal value of C for primal solution for validation set: {optimal_C}")
print(f"Testing accuracy on validation set for optimal C: {maxAccuracy}")

Optimal value of C for primal solution for validation set: 4
Testing accuracy on validation set for optimal C: 0.9748888888888889


# Finding the accuracy on test set

In [31]:
# Accuracy on test set
final_primal_model = svm_train_primal(X_train, y_train, optimal_C)
final_test_accuracy = svm_predict_primal(X_test, y_test, final_primal_model)
print(f"The test accuracy on test dataset. is {final_test_accuracy}")

The test accuracy on test dataset. is 0.9746666666666667


<h5 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Question 6</h5>

### Doing the same process as question 5 but with SVC package 
### After choosing optimal C we will use it to predict values for test set

In [32]:
X_test = X_test.reshape(-1, num_features)
X_val = X_val.reshape(-1, num_features)
C_list = [2**-10, 2**-8, 2**-6, 2**-4, 2**-2, 2**0, 2**2, 2**4, 2**6, 2**8, 2**10]
sklearn_test_accuracy_list = []
for c in C_list:
    sklearn_svm = svm.SVC(C = c/num_samples, kernel='linear')
    sklearn_svm.fit(X_train, y_train)
    sklearn_test_accuracy_list.append(sklearn_svm.score(X_val, y_val))
sklearn_max_accuracy = max(sklearn_test_accuracy_list)
bestAccuracyIdx = sklearn_test_accuracy_list.index(sklearn_max_accuracy)
optimalC = C_list[bestAccuracyIdx]

In [33]:
print(f"Best accuracy using SVC: {sklearn_max_accuracy}")
print(f"Best optimal C using SVC: {optimalC}")

Best accuracy using SVC: 0.9748888888888889
Best optimal C using SVC: 4


**Note**
- We got the optimal `C` as `4` and `accuracy` on `validation set` as `0.9748888888888889`
- This is same that we got for primal form on validation set

# Test accuracy on test dataset

- We put C = optimalC/num_samples because in the primal form, the formula was ingested with C/N
- To obtain similar results we are ingesting optimalC / num_samples

In [34]:
# Choosing optimal C as 4 
svm_instance = svm.SVC(C = optimalC / num_samples, kernel="linear")
svm_instance.fit(X_train, y_train)

SVC(C=0.001, kernel='linear')

In [35]:
# We can calculate accuracy using direct method score
accuracy_score = svm_instance.score(X_test, y_test)
print(f"Accuracy on test dataset using SVM.SVC is {accuracy_score}")

Accuracy on test dataset using SVM.SVC is 0.9746666666666667


In [36]:
# We can also try to predict y_pred using predict and then compute accuracy
def get_accuracy(y_pred, y_label):
    correct = 0
    total = len(y_label)
    for idx, p in enumerate(y_label):
        if p == y_pred[idx]:
            correct += 1
    return correct / total

In [37]:
y_preds = svm_instance.predict(X_test)

In [38]:
accuracy_score_custom = get_accuracy(y_preds, y_test)
print(f"Accuracy on test dataset using SVC class but custom accuracy method is {accuracy_score_custom}")

Accuracy on test dataset using SVC class but custom accuracy method is 0.9746666666666667


**NOTE**
- Both the approaches for calculating accuracy yiels same results